In [1]:
import datapreprocessing as dp
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
import functions as f
import time
import scanpy as sc
import pandas as pd
import pickle



# Which genes are actually significant? 
Looking at the curves for pca they peak at: 0.18 or 0.16 (see GeneAnalysis.ipynb)
Idea: look at all coefficients, let 100 regresions or so run, and then count. 
How did I calculate the mean before? dont know..

In [11]:
vdata = sc.read_h5ad('velocity_adata.h5ad')
sc.pp.scale(vdata, layer='Ms')
sc.pp.scale(vdata, layer='velocity')

In [44]:
ms = vdata[:, ['Klf2', 'Mcm3', 'Hif1a', 'Tcf7']].to_df('Ms')
ms['louvain'] = vdata[:, ['Klf2', 'Mcm3', 'Hif1a', 'Tcf7']].obs['louvain']
ms['sampleID'] = vdata[:, ['Klf2', 'Mcm3', 'Hif1a', 'Tcf7']].obs['sampleID']
ms.columns.tolist()

['Klf2', 'Mcm3', 'Hif1a', 'Tcf7', 'louvain', 'sampleID']

In [36]:
pd.concat([vdata[:, ['Klf2', 'Mcm3', 'Hif1a', 'Tcf7']].to_df('Ms'), 
vdata[:, ['Klf2', 'Mcm3', 'Hif1a', 'Tcf7']].obs['louvain']])

,0,Hif1a,Klf2,Mcm3,Tcf7
index,,,,,
AACTCTTCACGAGAGT-1-0,NaN,0.150851,-0.890570,1.730505,-0.276533
AAGACCTAGCGTGTCC-1-0,NaN,-0.677637,-0.674589,1.279034,-0.448905
AAACCTGAGCAGACTG-1-0,NaN,-0.250628,-0.910900,1.178916,-0.000281
AACTGGTTCGTCCAGG-1-0,NaN,1.501355,0.260586,1.289911,-0.169010
AACGTTGGTCAGAATA-1-0,NaN,-0.298497,-0.916975,1.221119,0.010500
...,...,...,...,...,...
TTTGCGCGTGGCAAAC-1-3,3,NaN,NaN,NaN,NaN
TTTACTGTCCTTTCGG-1-3,0,NaN,NaN,NaN,NaN
TTGTAGGGTTCTGTTT-1-3,4,NaN,NaN,NaN,NaN


index
AACTCTTCACGAGAGT-1-0    2
AAGACCTAGCGTGTCC-1-0    2
AAACCTGAGCAGACTG-1-0    2
AACTGGTTCGTCCAGG-1-0    2
AACGTTGGTCAGAATA-1-0    2
                       ..
TTTGCGCGTGGCAAAC-1-3    3
TTTACTGTCCTTTCGG-1-3    0
TTGTAGGGTTCTGTTT-1-3    4
TTTATGCTCAATACCG-1-3    3
TTGGCAAGTCTTCTCG-1-3    0
Name: louvain, Length: 13694, dtype: category
Categories (6, object): ['0', '1', '2', '3', '4', '5']

Okay. So random forest gives me back a feature importance, which is something I could look at and also permut permutations to find the significance. The oob error is comparable to CV-> dont really need to do CV, will just run the random forest 10 times and save everything ?

THis is fastest. Then can run CV individually on each gene, will be a sidestep from what I did before.. maybe need to pass: multioutput or something.. and then have an individual catch for rf to look for feature importance. 
theoretically we can also calculate the mean/sd... maybe do both? 
1. run eveything with only feature importance
2. compare models. look into making one better... 

In [7]:
coefs = pd.read_pickle('rf/Coefs.pkl')

In [10]:
f.help_pivot_to_df(coefs)


,target,predictors,coefficients
0,Mrpl15,Klf2,0.008825
1,Mrpl15,Mcm5,0.009117
2,Mrpl15,Mcm6,0.009936
3,Mrpl15,Mcm3,0.006627
4,Mrpl15,Ung,0.003514
...,...,...,...
4555,Mcm3,Tnfrsf9,0.002859
4556,Mcm3,Top2a,0.013315
4557,Mcm3,Ubc,0.003172
4558,Mcm3,Vim,0.003322


In [4]:
res

,fit_time,score_time,estimator,test_r2,train_r2,test_neg_mean_squared_error,train_neg_mean_squared_error,target
0,2.292749,0.812646,"(DecisionTreeRegressor(max_depth=10, max_featu...",0.325728,0.504561,-0.734672,-0.490358,Mrpl15
1,3.064834,0.245885,"(DecisionTreeRegressor(max_depth=10, max_featu...",0.338510,0.504940,-0.639841,-0.496817,Mrpl15
2,3.072310,0.241957,"(DecisionTreeRegressor(max_depth=10, max_featu...",0.317659,0.502015,-0.614285,-0.503367,Mrpl15
3,3.117134,0.280951,"(DecisionTreeRegressor(max_depth=10, max_featu...",0.305799,0.501510,-0.651011,-0.501896,Mrpl15
4,3.127586,0.259252,"(DecisionTreeRegressor(max_depth=10, max_featu...",0.335881,0.504899,-0.697075,-0.492325,Mrpl15
5,2.984620,0.274962,"(DecisionTreeRegressor(max_depth=10, max_featu...",0.280313,0.499595,-0.721662,-0.500209,Mrpl15
6,3.058965,0.297353,"(DecisionTreeRegressor(max_depth=10, max_featu...",0.344244,0.500443,-0.656717,-0.499409,Mrpl15
7,3.004831,0.166107,"(DecisionTreeRegressor(max_depth=10, max_featu...",0.315348,0.509684,-0.713369,-0.487990,Mrpl15
8,3.053746,0.085902,"(DecisionTreeRegressor(max_depth=10, max_featu...",0.291222,0.507047,-0.702963,-0.493357,Mrpl15
9,2.917250,0.104678,"(DecisionTreeRegressor(max_depth=10, max_featu...",0.323755,0.503513,-0.684550,-0.495740,Mrpl15


In [6]:
import numpy as np
depth = [t.estimators_[0].tree_.max_depth for t in res['estimator']]
depth

AttributeError: 'sklearn.tree._tree.Tree' object has no attribute 'min_depth'

In [156]:
[x[1] for x in list(zip(res['estimator'],res['target']))]

['Mrpl15',
 'Mrpl15',
 'Mrpl15',
 'Mrpl15',
 'Mrpl15',
 'Mrpl15',
 'Mrpl15',
 'Mrpl15',
 'Mrpl15',
 'Mrpl15',
 'Mybl1',
 'Mybl1',
 'Mybl1',
 'Mybl1',
 'Mybl1',
 'Mybl1',
 'Mybl1',
 'Mybl1',
 'Mybl1',
 'Mybl1',
 'Mcm3',
 'Mcm3',
 'Mcm3',
 'Mcm3',
 'Mcm3',
 'Mcm3',
 'Mcm3',
 'Mcm3',
 'Mcm3',
 'Mcm3']

In [30]:
import joblib

In [31]:
# imported and saved this. sounds good. 
joblib.dump(model, "./rftree/simple_rf_Mrpl15.joblib")

['./rftree/simple_rf_Mrpl15.joblib']

In [ ]:
#somehow the coefficeints, the dimensinos have to be transposed, pls.coef returns different dimensions. is this only for pca?
coefs, scores = f.generating_regressions(model, predictors, velocity_genes, X, y, 10, path = filepath, transpose_coefs=True)

coefs = f.help_pivot_to_df(coefs)

permut = f.evaluate_permutations(coefs, database_file, path = filepath)
end = time.time()

hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("THis is the time passed: {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))